# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import pickle

import torch.nn.functional as F
import numpy as np

from tqdm import tqdm
from torch import nn, optim

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
import pprint

with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

pprint.pprint(twits['data'][:15])

[{'message_body': '$FITB great buy at 26.00...ill wait',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:00:09Z'},
 {'message_body': '@StockTwits $MSFT',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:00:42Z'},
 {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a '
                  'rating of Hold setting target price at USD 350.00. Our own '
                  'verdict is Buy  http://www.stocktargetadvisor.com/toprating',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:01:24Z'},
 {'message_body': '$AMD I heard there’s a guy who knows someone who thinks '
                  'somebody knows something - on StockTwits.',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:01:47Z'},
 {'message_body': '$AMD reveal yourself!',
  'sentiment': 0,
  'timestamp': '2018-07-01T00:02:13Z'},
 {'message_body': '$AAPL Why the drop? I warren Buffet taking out his '
                  'position?',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:03:10Z'},
 {'message_body': '$BA bears have

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
pprint.pprint(messages[:15])
pprint.pprint(sentiments[:15])

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold '
 'setting target price at USD 350.00. Our own verdict is Buy  '
 'http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows '
 'something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention '
 'https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... '
 'if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% '
 'http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option stra

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [6]:
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    text = message.lower()

    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)',
        re.IGNORECASE
    )
    text = re.sub(url_pattern, ' ', text)
    
    symb_pattern = re.compile(r'[@#$][\w_-]+', re.IGNORECASE)
    text = re.sub(symb_pattern, ' ', text)
    
    not_alph_pattern = re.compile(r'[\W\d]')
    text = re.sub(not_alph_pattern, ' ', text)
    
    single_char = re.compile(r'\s[\w]\s')
    text = re.sub(single_char, ' ', text)
    
    tknizr = TweetTokenizer()
    tokenized_words = tknizr.tokenize(text)
    tokens = [w for w in tokenized_words]
    
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens if len(w) > 1]
    
    return tokens

# 13
# @DEEPAKM2013 @Nytunes Start of new Q on Monday. Expect strong buy volume across key companies of various sectors. $AMZN $AAPL
# 14
# [BREAKOUT Strategy] Current Portfolio : $ZBRA,$WEB,$TIF,$SRE,$SPPI,$OTEX,$OMF,$NVGS,$NRCIB,$MSG,$KIRK,$GHDX,$FBNK,$ESND,$DRI,$DKS,$CVTI,$CV

msg = messages[14]
preprocess(msg)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['breakout', 'strategy', 'current', 'portfolio']

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [7]:
tokenized = []

for twit in messages:
    tokenized.append(preprocess(twit))

In [8]:
print(tokenized[:10])

[['great', 'buy', 'at', 'ill', 'wait'], [], ['for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy'], ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['reveal', 'yourself'], ['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention'], ['ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol'], ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'], ['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', 'if', 'bo', 'then', 'bingo', 'what', 'is', 'the', 'odds']]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter

words = []

for msg in tokenized:
    words += msg
    
def bag_of_words(words):
    """Create a vocabulary by using Bag of words
    """
    bow = Counter()
    
    for word in words:
        bow[word] += 1
    return bow


bow = bag_of_words(words)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [10]:
# Printing to check data structure.

len_tokenized = len(tokenized)
len_bow = len(bow)
spcr = '\n' + '-'*15 + '\n'

print(f'{spcr}Length Tokenized Twits:{len_tokenized}')

for ndx, twit in enumerate(tokenized[:10]):
    print(f'\n{twit}')
great = bow['great']
print(f'{spcr}Length BOW:{len_bow}\n{great}')


---------------
Length Tokenized Twits:1548010

['great', 'buy', 'at', 'ill', 'wait']

[]

['for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy']

['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits']

['reveal', 'yourself']

['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position']

['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention']

['ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol']

['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above']

['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', 'if', 'bo', 'then', 'bingo', 'what', 'is', 'the', 'odds']

---------------
Length BOW:105283
17570


In [11]:
"""
Set the following variables:
    freqs = Dictionart that contains the Frequency of words appearing in messages. \
            The key is the token and the value is the frequency of that word in the corpus.
    low_cutoff = Float that is the frequency cutoff. Drop words with a frequency that is lower or equal \
            to this number.
    high_cutoff = Integer that is the cut off for most common words. Drop words that are the `high_cutoff` \
            most common words.
    K_most_common = The k most common words in the corpus. Use `high_cutoff` as the k.
"""

total_count = len(bow)
freqs = {word: count/total_count for word, count in bow.items()}

low_cutoff = 1e-5
high_cutoff = 10
K_most_common = bow.most_common(high_cutoff)

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(len(tokenized))
print(f'{spcr}{K_most_common}\n')
print(len(filtered_words)) 
print(f'{spcr}{filtered_words[:50]}')

1548010

---------------
[('the', 398828), ('to', 379789), ('amp', 295510), ('is', 284902), ('for', 273569), ('on', 241672), ('of', 211367), ('and', 208527), ('in', 205367), ('this', 203576)]

48577

---------------
['great', 'buy', 'at', 'ill', 'wait', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'usd', 'our', 'own', 'verdict', 'is', 'heard', 'there', 'guy', 'who', 'know', 'someone', 'think', 'somebody', 'something', 'on', 'stocktwits', 'reveal', 'yourself', 'why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position', 'bear', 'have', 'reason', 'to', 'pay', 'more', 'attention', 'ok']


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [12]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {}

for word_id, word in enumerate(filtered_words, 1):
    vocab[word] = word_id
    id2vocab[word_id] = word

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in tknzd_msg if word in filtered_words] for tknzd_msg in tokenized]

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [13]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [14]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19523199060467847

Finally let's convert our tokens into integer ids which we can pass to the network.

In [15]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']


------------------------
**⬇ CAN BE REMOVED ⬇**

### Notebook Objects
__Reloads Notebook Pyhton Objects__


In [16]:
def hydrator(d):
    """Keeps data fresh."""
    with open('data.pickle', 'wb+') as f:
        pickle.dump(d, f)
        f.close()

In [17]:
data = {
    'messages': messages,
    'sentiments': sentiments,
    'tokenized': tokenized,
    'bow': bow,
    'filtered_words': filtered_words,
    'vocab': vocab,
    'id2vocab': id2vocab,
    'filtered': filtered,
    'balanced': balanced,
    'token_ids': token_ids,
}

# hydrator(data)

In [18]:
def rehydrate_proj():
    """Keeps data fresh, without rerunning everythign above."""
    with open('data.pickle', 'rb') as f:
        data = pickle.load(f)
        messages = data['messages']
        sentiments = data['sentiments']
        tokenized = data['tokenized']
        bow = data['bow']
        filtered_words = data['filtered_words']
        vocab = data['vocab']
        id2vocab = data['id2vocab']
        filtered = data['filtered']
        balanced = data['balanced']
        token_ids = data['token_ids']
        return messages, sentiments, tokenized, bow, filtered_words, vocab, id2vocab, filtered, balanced, token_ids
        f.close()

In [19]:
messages, sentiments, tokenized, bow, filtered_words, vocab, id2vocab, filtered, balanced, token_ids = rehydrate_proj()

**⬆ CAN BE REMOVED ⬆**
___

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [20]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,
                            dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)
        self.logsm = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)

        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # Softmax function
        logps = self.logsm(out)

        batch_size = nn_input.size(1)
        
        return logps, hidden

### View Model

In [21]:
model = TextClassifier(vocab_size=len(vocab), embed_size=10, lstm_size=6, output_size=5, lstm_layers=2, dropout=0.1)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.4987, -1.4484, -1.5218, -1.5268, -2.2434],
        [-1.5015, -1.4520, -1.5142, -1.5303, -2.2382],
        [-1.5079, -1.4349, -1.5332, -1.5189, -2.2476],
        [-1.4943, -1.4554, -1.5128, -1.5255, -2.2584]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [22]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [23]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
split_frac = 0.8

split_idx = int(len(token_ids)*split_frac)
train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

In [24]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(48578, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logsm): LogSoftmax()
)

In [26]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 512
learning_rate = 1e-3
clip = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    steps = 0
    
    tqdm_dataloader = tqdm(
        dataloader(train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True)
    )
    
    for text_batch, labels in tqdm_dataloader:
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        # ------------------------------
        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, hidden = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        # ------------------------------
        tqdm_dataloader.set_description(f'Epoch:{epoch + 1} | Batch:{steps} | Training Loss:{loss:.3f}')
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            # Print the training loss, validation loss, and validation accuracy for every 100 steps.
            # ------------------------------
            validation_losses = []        
            
            
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):
                inputs, labels = inputs.to(device), labels.to(device)
                
            valid_hidden = model.init_hidden(labels.shape[0])
            
            for each in valid_hidden:
                each.to(device)
            output, valid_hidden = model(inputs, valid_hidden)
            
            validation_loss = criterion(output.squeeze(), labels)
            validation_losses.append(validation_loss.item())
            
            model.train()
            # ------------------------------
            tqdm.write(
                f'Epoch:{epoch + 1}/{epochs} Batch:{steps} | Validation Loss:{np.mean(validation_losses):.3f}'
            )

Epoch:1 | Batch:101 | Training Loss:0.984: 101it [00:24,  1.38it/s]

Epoch:1/3 Batch:100 | Validation Loss:0.813


Epoch:1 | Batch:200 | Training Loss:0.908: 200it [00:46,  1.03it/s]

Epoch:1/3 Batch:200 | Validation Loss:0.715


Epoch:1 | Batch:301 | Training Loss:0.890: 301it [01:09,  1.37it/s]

...

Epoch:3 | Batch:1500 | Training Loss:0.580: 1500it [05:50,  1.04it/s]

Epoch:3/3 Batch:1500 | Validation Loss:0.533


Epoch:3 | Batch:1600 | Training Loss:0.557: 1600it [06:14,  1.05it/s]

Epoch:3/3 Batch:1600 | Validation Loss:0.544


Epoch:3 | Batch:1612 | Training Loss:0.604: 1612it [06:16,  4.55it/s]


## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [31]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Convert words to ids
    tokens = [vocab[word] if word is not None else "sell" for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1)))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [32]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0001,  0.0084,  0.0036,  0.8727,  0.1152]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Model scores or output are it's predicted probability distribution, all values adding to one, of the given message to the sentiment classes.  Each score or probabilty is the confidence of the model that the twit sentiment falls within that sentiment class.  For this twit, the prediction of the model is that the sentiment of the twit is positive with a score of 0.89667 in the second to last class position.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [33]:
import pprint
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)
    td = test_data['data'][150:200]
    pprint.pprint(td)

[{'message_body': '$ORLY break 280 then free fall from there',
  'timestamp': '2018-11-01T00:30:48Z'},
 {'message_body': '@MapTechnicalForecasting But what will S&amp;P500 be at Fri '
                  'lunchtime after $AAPL reports Thurs afternoon? That is the '
                  'more relevant question',
  'timestamp': '2018-11-01T00:30:55Z'},
 {'message_body': '$EBAY earnings move  5.9% vs an expected ±9.7% move  '
                  'http://tinyurl.com/y7rldku9',
  'timestamp': '2018-11-01T00:30:59Z'},
 {'message_body': '$CRM max pain is 139 for expiry 2018-11-02 Source: '
                  'http://sweep.ly/maxpain.html',
  'timestamp': '2018-11-01T00:31:11Z'},
 {'message_body': '$GE Firm market cap about $88B.  Need to time 25-1 reverse '
                  'split when she hits $8.75 this Q.',
  'timestamp': '2018-11-01T00:31:15Z'},
 {'message_body': '$ABMD I’m afraid to hold during earning so sold both call '
                  'and put today: will buy back after earning look at $is

### Twit Stream

In [34]:
def twit_stream():
    for twit in td:
        yield twit

Using the `prediction` function, let's apply it to a stream of twits.

In [35]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        print(text)
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [36]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

for i in range(15):
    pred = next(score_stream)
    if pred:
        print(pred)
        print('-----------------\n')
    else:
        print("BOOM!")

$ORLY break 280 then free fall from there
@MapTechnicalForecasting But what will S&amp;P500 be at Fri lunchtime after $AAPL reports Thurs afternoon? That is the more relevant question
{'symbol': '$AAPL', 'score': tensor([[ 0.0156,  0.0521,  0.5695,  0.3384,  0.0243]]), 'timestamp': '2018-11-01T00:30:55Z'}
-----------------

$EBAY earnings move  5.9% vs an expected ±9.7% move  http://tinyurl.com/y7rldku9
$CRM max pain is 139 for expiry 2018-11-02 Source: http://sweep.ly/maxpain.html
$GE Firm market cap about $88B.  Need to time 25-1 reverse split when she hits $8.75 this Q.
$ABMD I’m afraid to hold during earning so sold both call and put today: will buy back after earning look at $isrg as a study case after war
$BAC Banking sector is about to soar. Banks spends so much money trying to comply with the rules that were placed after the financial crisis
$BAC back in for like no ride now...
$AMD $DJIA  Happy Halloween From THE STAGGERRRRR......
 https://www.youtube.com/watch?v=2x-4BAqk4wc
$

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.